<a href="https://colab.research.google.com/github/hans321/CS4240/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [4]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-03 09:11:26--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-03 09:11:27 (92.7 MB/s) - ‘utils.py’ saved [4519/4519]



In [5]:
#try:
#    from Models import *
#except ModuleNotFoundError: 
    #!wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
#    !wget https://github.com/hans321/CS4240/raw/master/Models.py
#    from Models import *
!wget https://github.com/hans321/CS4240/raw/master/Models.py
from Models import *

--2020-03-03 09:16:16--  https://github.com/hans321/CS4240/raw/master/Models.py
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hans321/CS4240/master/Models.py [following]
--2020-03-03 09:16:16--  https://raw.githubusercontent.com/hans321/CS4240/master/Models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3007 (2.9K) [text/plain]
Saving to: ‘Models.py’

Models.py           100%[===================>]   2.94K  --.-KB/s    in 0s      

2020-03-03 09:16:16 (75.1 MB/s) - ‘Models.py’ saved [3007/3007]



In [16]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [0]:
model = ModelC_AllConvNet(3)

if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [19]:
for epoch in range(3):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.301215
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.303068
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.301723
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.299967
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.301508
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.303333
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.304112
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.302154
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.302207
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.302731
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.303431
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.302804
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.304833
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.302622
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.302913
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.301957

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:01<00:00, 86.80it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303912
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.302955
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.302686
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.302240
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.302685
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.302716
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.302677
Train Epoch: 1 [22400/50000 (45%)]	Loss: 2.302163
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.302877
Train Epoch: 1 [28800/50000 (58%)]	Loss: 2.302503
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.302243
Train Epoch: 1 [35200/50000 (70%)]	Loss: 2.302277
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.303004
Train Epoch: 1 [41600/50000 (83%)]	Loss: 2.302666
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.302585
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.302589

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:01<00:00, 90.33it/s]


Train Epoch: 2 [0/50000 (0%)]	Loss: 2.302593
Train Epoch: 2 [3200/50000 (6%)]	Loss: 2.303027
Train Epoch: 2 [6400/50000 (13%)]	Loss: 2.302489
Train Epoch: 2 [9600/50000 (19%)]	Loss: 2.301708
Train Epoch: 2 [12800/50000 (26%)]	Loss: 2.302694
Train Epoch: 2 [16000/50000 (32%)]	Loss: 2.302871
Train Epoch: 2 [19200/50000 (38%)]	Loss: 2.302586
Train Epoch: 2 [22400/50000 (45%)]	Loss: 2.302583
Train Epoch: 2 [25600/50000 (51%)]	Loss: 2.302582
Train Epoch: 2 [28800/50000 (58%)]	Loss: 2.302582
Train Epoch: 2 [32000/50000 (64%)]	Loss: 2.302584
Train Epoch: 2 [35200/50000 (70%)]	Loss: 2.302575
Train Epoch: 2 [38400/50000 (77%)]	Loss: 2.302585
Train Epoch: 2 [41600/50000 (83%)]	Loss: 2.302581
Train Epoch: 2 [44800/50000 (90%)]	Loss: 2.302586
Train Epoch: 2 [48000/50000 (96%)]	Loss: 2.302584

Test set: Average loss: 0.0187, Accuracy: 1003/10000 (10%)



In [0]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")